In [1]:
!pip install watson_developer_cloud

In [2]:
!pip install --upgrade "ibm-watson>=4.5.0"

Requirement already up-to-date: ibm-watson>=4.5.0 in c:\users\bhushan\anaconda3\lib\site-packages (4.5.0)


In [3]:
import pandas as pd
import numpy as np
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions,SentimentOptions
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [4]:
authenticator = IAMAuthenticator('Z0U9sUFcokSGxve8YyGjMF30jRyAM1zFLLZMUHMEpZFL')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator)

natural_language_understanding.set_service_url('https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/3decfb57-67b4-46cf-9f99-426006945062')

In [5]:
data = pd.read_csv('Lockdown1.0labelled.csv')

In [6]:
df = data[['tweet']]

In [7]:
df = df[:50]
df

,tweet
0,the jio company launched their new plans calle...
1,reliance jio offers free internet airtel voda ...
2,yes home only you can work from home you are
3,apart from the jokes you may you need anyone t...
4,work from home
5,this the time protect ourselves from covid thi...
6,yes these are mine putting kapoor not kanika r...
7,prevent them coming out home local nagar nigam...
8,put home quarantine stamp hands take aadhaar p...
9,just want ask there’ corresponding fine those ...


In [14]:
def sentiment_and_keyword(st,service=natural_language_understanding):
    return (service.analyze(text=st,language='en',features=Features(keywords=KeywordsOptions(sentiment= True,emotion= True,limit= 2),sentiment=SentimentOptions())).get_result())

In [15]:
df['sentiment_keyword_json'] = df.apply(lambda row : sentiment_and_keyword(str(row['tweet'])),axis = 1)

In [18]:
df2 = df[['tweet','sentiment_keyword_json']]

In [19]:
"""
Extract sentiment score and append it to a new column in the dataframe
INPUT: JSON
OUTPUT: NUMERIC SCOREs
"""
def sentiment_extractor(row_num):
    sentiment_scores=df2['sentiment_keyword_json'][row_num]['sentiment']['document']
    return sentiment_scores
    
"""
Extract keywoord score/s and append it to a new column in the dataframe
INPUT: JSON
OUTPUT: DICTIONARY [{Keyword1:Score},{Keyword2:Score},{Keyword3:Score}]
"""
def keyword_extractor(row_num):
    keyword_scores=[{df2['sentiment_keyword_json'][row_num]['keywords'][i]['text'] : df2['sentiment_keyword_json'][row_num]['keywords'][i]['sentiment']['score']} for i in range(0,len(df2['sentiment_keyword_json'][row_num]['keywords']))]
    return keyword_scores

In [20]:
df2['sentiment_score'] = df2.apply(lambda row : sentiment_extractor(row.name),axis = 1)
df2['keyword_score'] = df2.apply(lambda row : keyword_extractor(row.name),axis = 1)
df2

,tweet,sentiment_keyword_json,sentiment_score,keyword_score
0,the jio company launched their new plans calle...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}","[{'jio company': 0}, {'work': 0}]"
1,reliance jio offers free internet airtel voda ...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.845686, 'label': 'positive'}","[{'reliance jio': 0.845686}, {'work': 0.845686}]"
2,yes home only you can work from home you are,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}",[{'home': 0}]
3,apart from the jokes you may you need anyone t...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.904985, 'label': 'positive'}","[{'jokes': 0.904985}, {'lockdown': 0.904985}]"
4,work from home,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}",[{'home': 0}]
5,this the time protect ourselves from covid thi...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.769895, 'label': 'positive'}","[{'stage people': 0.769895}, {'best cure quest..."
6,yes these are mine putting kapoor not kanika r...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.64238, 'label': 'positive'}","[{'kanika rana': 0.64238}, {'different corners..."
7,prevent them coming out home local nagar nigam...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}","[{'local nagar nigam workers': 0}, {'door door..."
8,put home quarantine stamp hands take aadhaar p...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': -0.787607, 'label': 'negative'}","[{'home quarantine stamp hands': -0.787607}, {..."
9,just want ask there’ corresponding fine those ...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.788989, 'label': 'positive'}","[{'home regulations thanks': 0.788989}, {'comp..."


In [21]:
import json
from pandas.io.json import json_normalize

In [22]:
df2[['score','label']]=json_normalize(data=df2['sentiment_score'],max_level=1)

In [23]:
df2

,tweet,sentiment_keyword_json,sentiment_score,keyword_score,score,label
0,the jio company launched their new plans calle...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}","[{'jio company': 0}, {'work': 0}]",0.000000,neutral
1,reliance jio offers free internet airtel voda ...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.845686, 'label': 'positive'}","[{'reliance jio': 0.845686}, {'work': 0.845686}]",0.845686,positive
2,yes home only you can work from home you are,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}",[{'home': 0}],0.000000,neutral
3,apart from the jokes you may you need anyone t...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.904985, 'label': 'positive'}","[{'jokes': 0.904985}, {'lockdown': 0.904985}]",0.904985,positive
4,work from home,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}",[{'home': 0}],0.000000,neutral
5,this the time protect ourselves from covid thi...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.769895, 'label': 'positive'}","[{'stage people': 0.769895}, {'best cure quest...",0.769895,positive
6,yes these are mine putting kapoor not kanika r...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.64238, 'label': 'positive'}","[{'kanika rana': 0.64238}, {'different corners...",0.642380,positive
7,prevent them coming out home local nagar nigam...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0, 'label': 'neutral'}","[{'local nagar nigam workers': 0}, {'door door...",0.000000,neutral
8,put home quarantine stamp hands take aadhaar p...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': -0.787607, 'label': 'negative'}","[{'home quarantine stamp hands': -0.787607}, {...",-0.787607,negative
9,just want ask there’ corresponding fine those ...,"{'usage': {'text_units': 1, 'text_characters':...","{'score': 0.788989, 'label': 'positive'}","[{'home regulations thanks': 0.788989}, {'comp...",0.788989,positive


In [35]:
data = data[:50]
data

,Unnamed: 0,Unnamed: 0.1,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,tokens,predictions
0,0,0,1242879759601799168,1242745110661435393,1585160600000,2020-03-25 23:53:20,530,NaN,the jio company launched their new plans calle...,[],...,NaN,NaN,NaN,"[{'user_id': '853512722113323008', 'username':...",NaN,NaN,NaN,NaN,"['jio', 'company', 'launched', 'new', 'plans',...",0
1,1,1,1242879607176478720,1242879607176478720,1585160564000,2020-03-25 23:52:44,530,NaN,reliance jio offers free internet airtel voda ...,[],...,NaN,NaN,NaN,"[{'user_id': '869018357432950785', 'username':...",NaN,NaN,NaN,NaN,"['reliance', 'jio', 'offers', 'free', 'interne...",1
2,2,2,1242879202866515969,1242450477779800065,1585160468000,2020-03-25 23:51:08,530,NaN,yes home only you can work from home you are,[],...,NaN,NaN,NaN,"[{'user_id': '804045415322554368', 'username':...",NaN,NaN,NaN,NaN,"['yes', 'home', 'work', 'home']",1
3,3,3,1242878275585802240,1242873807137726464,1585160247000,2020-03-25 23:47:27,530,NaN,apart from the jokes you may you need anyone t...,[],...,NaN,NaN,NaN,"[{'user_id': '714465124782768128', 'username':...",NaN,NaN,NaN,NaN,"['apart', 'jokes', 'may', 'need', 'anyone', 't...",0
4,4,4,1242876746459803658,1242717249590321157,1585159882000,2020-03-25 23:41:22,530,NaN,work from home,[],...,NaN,NaN,NaN,"[{'user_id': '276394575', 'username': 'hiren_r...",NaN,NaN,NaN,NaN,"['work', 'home']",0
5,5,5,1242876655770562560,1242873284372369410,1585159860000,2020-03-25 23:41:00,530,NaN,this the time protect ourselves from covid thi...,[],...,NaN,NaN,NaN,"[{'user_id': '1237698695748923393', 'username'...",NaN,NaN,NaN,NaN,"['time', 'protect', 'covid', 'stage', 'people'...",1
6,6,6,1242874426531516421,1242868816922791936,1585159329000,2020-03-25 23:32:09,530,NaN,yes these are mine putting kapoor not kanika r...,"['#lockdown21', '#ahmedabadlockdown']",...,NaN,NaN,NaN,"[{'user_id': '257816770', 'username': 'jaladhi...",NaN,NaN,NaN,NaN,"['yes', 'mine', 'putting', 'kapoor', 'kanika',...",1
7,7,7,1242874107504422912,1242874105461788672,1585159253000,2020-03-25 23:30:53,530,"Shivalik nagar, haridwar",prevent them coming out home local nagar nigam...,[],...,NaN,NaN,NaN,"[{'user_id': '1242867651484975104', 'username'...",NaN,NaN,NaN,NaN,"['prevent', 'coming', 'home', 'local', 'nagar'...",1
8,8,8,1242873841363251207,1242862753813061632,1585159189000,2020-03-25 23:29:49,530,NaN,put home quarantine stamp hands take aadhaar p...,['#coronavirusindia'],...,NaN,NaN,NaN,"[{'user_id': '797761076595068928', 'username':...",NaN,NaN,NaN,NaN,"['put', 'home', 'quarantine', 'stamp', 'hands'...",0
9,9,9,1242873018122145795,1242873018122145795,1585158993000,2020-03-25 23:26:33,530,NaN,just want ask there’ corresponding fine those ...,[],...,NaN,NaN,NaN,"[{'user_id': '2410837254', 'username': 'Charmi...",NaN,NaN,NaN,NaN,"['want', 'ask', 'there’', 'corresponding', 'fi...",0


In [53]:
pos = df2['label']=='positive'
neg = df2['label']=='negative'
pos1 = data['predictions'] == 0
neg1 = data['predictions'] == 1

In [54]:
comp = df2[neg]
comp1 = data[neg1]

In [55]:
comp = comp[:10]
comp1 = comp1[:10]

In [62]:
comp[['tweet','label']]

,tweet,label
8,put home quarantine stamp hands take aadhaar p...,negative
11,you revoke article 370 become silent you shutd...,negative
12,suffering with very bad internet connection ve...,negative
13,first time doesn want leave from office first ...,negative
15,wow this all video suggest that corona has ask...,negative
18,stay indoor stay safe just the way this spidey...,negative
19,finally carelessness has gone make wiseness hi...,negative
21,has not announced paid holidays possible emplo...,negative
22,everytime think would work hathway always disa...,negative
23,wonder why shutting down clearly the worst net...,negative


In [57]:
comp1['predictions']

1     1
2     1
5     1
6     1
7     1
10    1
14    1
15    1
17    1
20    1
Name: predictions, dtype: int64

In [58]:
comp['label']

8     negative
11    negative
12    negative
13    negative
15    negative
18    negative
19    negative
21    negative
22    negative
23    negative
Name: label, dtype: object

In [59]:
df2['label']

0      neutral
1     positive
2      neutral
3     positive
4      neutral
5     positive
6     positive
7      neutral
8     negative
9     positive
10     neutral
11    negative
12    negative
13    negative
14     neutral
15    negative
16     neutral
17     neutral
18    negative
19    negative
20     neutral
21    negative
22    negative
23    negative
24    positive
25     neutral
26    positive
27    negative
28    positive
29     neutral
30    negative
31    positive
32    negative
33    negative
34    negative
35    negative
36    negative
37    positive
38    positive
39    positive
40     neutral
41    negative
42    positive
43    negative
44    negative
45    negative
46    negative
47     neutral
48     neutral
49     neutral
Name: label, dtype: object